In [1]:
%matplotlib inline
import pandas as pd
import os
import json
import pyap
import matplotlib.pyplot as plt
import seaborn as sns

from elasticsearch import (
    Elasticsearch,
    helpers
)
from datetime import datetime
from io import StringIO
from configparser import ConfigParser
from sqlalchemy import create_engine


data_dir = os.path.join(os.pardir,'data')
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

es_creds = get_ini_vals(config_file, 'elasticsearch')
mysql_creds = get_ini_vals(config_file, 'mysql')

### Connect to ES


Newest indices are formatted like

`<issue_name>-YY.MM.DD`

In [2]:
es = Elasticsearch(
    [es_creds['host']],
    http_auth=('',''),
    port = es_creds['port'],
    use_ssl=False
)
print(es.info())

# print all indices
indices=es.indices.get_alias().keys()
print(indices)

{'cluster_name': 'elasticsearch', 'version': {'build_hash': 'db0d481', 'build_snapshot': False, 'number': '5.2.1', 'lucene_version': '6.4.1', 'build_date': '2017-02-09T22:05:32.386Z'}, 'tagline': 'You Know, for Search', 'name': 'pvnM2mO', 'cluster_uuid': 'lf55i5JCSBq8nk_ZrTyTKQ'}
dict_keys(['climate-17.03.23', 'civil_right-17.03.20', '17.03.08', '17.03.06', 'healthcare-17.03.18', '17.03.14', '17.03.07', 'climate-17.03.22', 'speech-17.03.23', 'type', 's-17.03.09', 'n-17.03.18', 'civil_rights-17.03.21', 'twitter-ross', 't-test', 'speech-17.03.22', 'healthcare-17.03.19', 'womens_right-17.03.22', 'twitter', 'lgbt-17.03.23', 'healthcare-17.03.20', 'civil_right-17.03.18', 'speech-17.03.21', 'shakespeare', '17.03.09', '17.03.12', 'voting-17.03.22', '17.03.10', '17.03.11', '17.03.17', 'climate-17.03.21', 'voting-17.03.23', 'immigrants-17.03.21', 's-17.03.10', 'civil_rights-17.03.23', 'worker-17.03.22', '17.03.16', '17.03.13', 'worker-17.03.23', '.kibana', 'womens_right-17.03.23', 'voting-17.03

### Connect To Rszt MySQL

In [4]:
# connect to openatrium database
engine = create_engine(
    """mysql+pymysql://{user}:{password}@{host}:{port}/{db}"""
    .format(user=mysql_creds['user'],
            password=mysql_creds['password'],
            host=mysql_creds['host'],
            port=mysql_creds['port'],
            db=mysql_creds['database']
            )
                      )

conn = engine.connect()

# table fields
rzst_event_header =[
    'elasticsearch_id',
    'title',
    'body_value',
    'location_text',
    'event_datetime_from',
    'event_datetime_to',
    'score',
    'insert_dt',
    'event_type',
    'pri_action_type',
    'sec_action_type'
]

# need to create a flow that parses out data we need first...
insert_vals= dict.fromkeys(rzst_event_header, "")

### Run Set of queries, plot scores of results


We have two main indices in Elasticsearch that are issue-specific

In [3]:
issues = ['civil_right', 
          'healthcare', 
          'voting', 
          'speech', 
          'immigrants',
          'lgbt',
          'worker',
          'womens_right'       
         ]
actions = {'charity': ["donat*",
                       "give",
                       "donate",
                       "give support",
                       "financial support"
                      ],
           'protest': ['protest', 
                       'march'
                      ],
           'petition': ['petition', 
                        'sign', 
                        'call'
                       ],
           'gathering': ["meetup",
                         "huddle",
                         "congregate",
                         "join us"
                        ],
           'boycott': ["boycott"],
           'advocate': ["call",
                        "email",
                        "reach out",
                        "senator",
                        "representative",
                        "sign petition",
                        "petition"
                       ],
           'vote': ["vote",
                    "cast your ballot"],
           'townhall': ["town hall",
                        "open office",
                        "town meeting",
                        "townhall",
                        'virtual townhall']
          }

In [13]:
# get number of results for one query
format = "%y.%m.%d"
index_ = issues[0] + "-" + datetime.now().date().strftime(format)
print(index_)

civil_right-17.03.19


In [50]:
# form query for one action
query = 'text: "' +  '" OR "'.join(actions['gathering']) + '"'
print(query)

text: "meetup" OR "huddle" OR "congregate" OR "join us"


In [51]:
results = es.search(index=index_,
           q=query,
           size=10000,    # have to manually set this, default is 10
           request_timeout=30)

In [52]:
import json

n_results = len(results['hits']['hits'])
for hit in results['hits']['hits']:
    print(hit['_score'], hit['_source'], hit['_id'])
    print("TEXT: %s \n" % hit['_source']['message'])
    print("USER: %s \n" % hit['_source']['user'])
    print("TIMESTAMP: %s \n" % hit['_source']['@timestamp'])

8.083777 {'urls': ['http://goo.gl/XGy51p', ''], '@timestamp': '2017-03-19T18:58:12.000Z', '@version': '1', 'tags': ['civil right'], 'source': 'http://twitter.com/exa_lemon/status/843537105544515584', 'symbols': [], 'user': 'exa_lemon', 'hashtags': [], 'client': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'message': 'RT @jennymizkz: Local horny women are waiting to meet you online! dont wait, join us 100% Free right now:  https://t.co/0AALAfd2lo… https://t.co/tNbWadVlrp', 'type': 'civil_right', 'user_mentions': [{'id_str': '784695352767250432', 'indices': [3, 14], 'id': 784695352767250432, 'name': 'jenny paek', 'screen_name': 'jennymizkz'}], 'retweeted': False} AVrn87eUMNAj8foZ-HYB
TEXT: RT @jennymizkz: Local horny women are waiting to meet you online! dont wait, join us 100% Free right now:  https://t.co/0AALAfd2lo… https://t.co/tNbWadVlrp 

USER: exa_lemon 

TIMESTAMP: 2017-03-19T18:58:12.000Z 

7.7183886 {'urls': ['http://goo.gl/jkasVh', ''], '@timestamp': '

In [45]:
n_results

0

In [55]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 1.8MB/s ta 0:00:01


In [4]:
import boto3
import csv
import os

from tqdm import tqdm


bucket = "mids-capstone-rzst"
session = boto3.Session(profile_name="berkeley")
s3 = session.client("s3", "us-west-2")

class TwitterQueryAction(object):
    def __init__(self, issue_key):
        self.issue = issue_key
        self.firstline = True
        self.outfile = self.issue + ".csv"
        self.buffer = open(self.outfile, "w")
        self.key_prefix = "/".join(["es_queries", datetime.now().date().isoformat()])
        self.line_count = 0
        
        
    @property
    def query(self):
        return 'message: "' +  '" OR "'.join(actions[self.action]) + '"'
    
    
    @property
    def s3_loc(self):
        return "/".join([self.key_prefix, self.outfile])
    
    
    def _write_tweets(self, results):
        if self.firstline:
            self.firstline = False
            fieldnames = ['issue',
                          'action',
                          'id',
                          'es_score',
                          'tweet_timestamp',
                          'query_timestamp',
                          'tweet_user',
                          'tweet'
                         ]
            self.writer = csv.DictWriter(self.buffer, 
                                        fieldnames=fieldnames
                                       )
            self.writer.writeheader()
        n_results = len(results['hits']['hits'])
        if n_results > 0:
            print("Writing %s results.\n" % n_results)
            for result in tqdm(results['hits']['hits']):
                row = {'issue': self.issue,
                       'action': self.action,
                       'id': result['_id'],
                       'es_score': result['_score'],
                       'tweet_timestamp': result['_source']['@timestamp'],
                       'query_timestamp': self.querytimestamp,
                       'tweet_user': result['_source']['user'],
                       'tweet': result['_source']['message']
                      }
                
                # skip retweets
                if row['tweet'][:2] == "RT":
                    continue
                self.writer.writerow(row)
                self.line_count += 1
        else:
            print("No results to save!")
            return None
        
    
    def run(self, action_key):
        self.action = action_key
        indices=[index for index in es.indices.get_alias().keys() if self.issue in index]
        self.querytimestamp = datetime.now().isoformat()
        for index_ in indices:
            print("Querying index %s" % index_)
            print(self.query)
            results = es.search(index=index_,
                                q=self.query,
                                size=10000,    # have to manually set this, default is 10
                                request_timeout=30)
            self._write_tweets(results)
        
    def stop(self):
        self.buffer.close()
        print("Uploading saved {line_count} results to s3://{bucket}/{s3_loc}".format(
                line_count = self.line_count,
                bucket = bucket,
                s3_loc = self.s3_loc
                )
             )
        s3.upload_file(self.outfile, bucket, "/".join([self.s3_loc
                                                      ])
                      )
        os.remove(self.outfile)

In [5]:
for issue in issues:
    tq = TwitterQueryAction(issue)

    for action in actions.keys():
        tq.run(action)
    tq.stop()

print("Done!")

Querying index civil_right-17.03.20
message: "vote" OR "cast your ballot"


100%|██████████| 84/84 [00:00<00:00, 70591.37it/s]

Writing 84 results.

Querying index civil_rights-17.03.21
message: "vote" OR "cast your ballot"
No results to save!
Querying index civil_right-17.03.18
message: "vote" OR "cast your ballot"



100%|██████████| 13/13 [00:00<00:00, 19091.72it/s]

Writing 1 results.

Querying index civil_rights-17.03.23
message: "vote" OR "cast your ballot"
Writing 13 results.

Querying index civil_right-17.03.19
message: "vote" OR "cast your ballot"


...


100%|██████████| 11/11 [00:00<00:00, 20289.07it/s]

Writing 11 results.

Querying index womens_right-17.03.23
message: "town hall" OR "open office" OR "town meeting" OR "townhall" OR "virtual townhall"



100%|██████████| 5/5 [00:00<00:00, 21013.55it/s]

Writing 5 results.

Uploading saved 3527 results to s3://mids-capstone-rzst/es_queries/2017-03-23/womens_right.csv


Done!
